# Perceptron, SVM, and Neural Networks

In [1]:
# imports
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

%matplotlib inline

# 1. Prepare the dataset

In [2]:
# import the assigned dataset
df = pd.read_csv('telescope.csv')
del df['Unnamed: 0']

In [3]:
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,targets
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,g


In [4]:
df.columns[-1]

'targets'

In [3]:
target_list = []
for row in df['targets']:
    if row == 'g':
        target_list.append(float(0))
    else:
        target_list.append(float(1))

In [4]:
df['targets_float'] = target_list

In [7]:
# Preprocess the dataset

df_x = df[["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans"]]
#df_x = df.columns[:-1]

df_y = df[['targets_float']]

In [8]:
# Make a train-test split with 80% to training and 20% to testing.
from sklearn.model_selection import train_test_split
features = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans"]
telescopex_train, telescopex_test, telescopey_train, telescopey_test = train_test_split( df_x,
                                                                        df_y, test_size=0.2, random_state = 4)#, stratify = df_mushrooms_v2['class'])


In [9]:
# Normalize numerical features and encode the categorical features (if any)
train_x_std = StandardScaler().fit_transform(telescopex_train)
test_x_std = StandardScaler().fit_transform(telescopex_test)



In [10]:
train_y = telescopey_train
test_y = telescopey_test

As this is a classification problem, the target variables must be categorical. Using the feature and target variable information, preprocess the dataset to use given features.

This step might require students to scale the features, one-hot encode categorical FEATURES (if any).



For feature scaling and one-hot encoding, go through:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

# 2: Creating a perceptron model for the processed dataset

The perceptron.py file in the resources provides functions to code the perceptron model from scratch.

Using the file as reference, write the functions:

1. cross_validation_split
2. accuracy_metric
3. evaluate_algorithm
4. predict
5. train_weights
6. perceptron

This step is aimed at providing a comprehensive understanding of the internal functioning of a perceptron model.

In [71]:
# your code for step 2

from random import randrange
# Split a dataset into k folds

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    #print("dataset------------",dataset_split)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    #print("evaluate_algo---------folds",  type(folds))
    for fold in folds:
        train_set = list(folds)
        #print("train_set ---------",  train_set )
        #print("\n fold ---------",  fold)
        train_set.remove(fold)
        #print("\n train_set after remove ---------",  fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
    #print(train, "train_weight-----------")
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
    return weights

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
    #print("perceptron----" , "train-----------", train)
    #print("perceptron----" , "test-----------", test)
    predictions = list()
    #print(train, "perceptron --------")
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return(predictions)

# 3: Batch size for perceptron model

Experiment with different batch sizes in the perceptron model (eg: 1, 4, 8).


In [72]:
# code for step 3
n_folds1 = 3
l_rate = 0.01
n_epoch = 500
dataset = np.hstack((train_x_std, train_y))
dataset = dataset.tolist()
scores = evaluate_algorithm(dataset, perceptron, n_folds1, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [69.61750788643532, 64.43217665615141, 68.88801261829653]
Mean Accuracy: 67.646%


In [98]:
# code for step 3
n_folds2 = 5
l_rate = 0.01
n_epoch = 500
dataset = np.hstack((train_x_std, train_y))
dataset = dataset.tolist()
scores = evaluate_algorithm(dataset, perceptron, n_folds2, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [61.715412421952024, 62.8984554715741, 55.14295103516267, 58.166283273085774, 44.6927374301676]
Mean Accuracy: 56.523%


In [99]:
# code for step 3

n_folds2 = 7
l_rate = 0.01
n_epoch = 500
dataset = np.hstack((train_x_std, train_y))
dataset = dataset.tolist()
scores = evaluate_algorithm(dataset, perceptron, n_folds2, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [64.88725264611136, 60.51541647491947, 57.57017947537966, 63.368614818223655, 61.84997699033594, 61.71191900598251, 62.724344224574324]
Mean Accuracy: 61.804%


# 4. SVM's

### **Note : You are allowed to use sklearn's SVC classifier for steps 4.1 through 4.3**

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

---

# 4.1 Linear SVM


---


# Step 1: Implement a linear SVM model to classify the data points. (Look into the 'kernel' parameter).

In [25]:
# linear SVM
from sklearn.svm import SVC
from sklearn import datasets

In [26]:
svm_clf = SVC(kernel="linear")

# Step 2: Train the model

In [27]:
# training - linear SVM
svm_clf.fit(train_x_std, train_y.values.ravel())

SVC(kernel='linear')

# Step 3: Predict for the test points using the model trained in the previous step

In [28]:
# predict - linear SVM
linear_svm_pred = svm_clf.predict(test_x_std)

# 4.2 Kernel SVM - Polynomial kernel

---



# Step 1: Implement a kernel SVM model with a polynomial kernel to classify the data points.

In [29]:
# kernel SVM - polynomial kernel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler


#create a pipeline to create features, scale data and fit the model
polynomial_svm_clf = Pipeline((
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scalar", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=10, coef0=1, C=5)) 
))


# Step 2: Train the model

In [30]:
# training - kernel SVM
polynomial_svm_clf.fit(train_x_std, train_y.values.ravel())

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scalar', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, degree=10, kernel='poly'))])

# Step 3: Predict for the test points using the model trained in the previous step

In [31]:
# predict - kernel SVM
poly_svm_pred = polynomial_svm_clf.predict(test_x_std)

# 4.3 Kernel SVM - Gaussian kernel

---



# Step 1: Implement a kernel SVM model with a gaussian (Radian Basis function) kernel to classify the data points.

In [32]:
# kernel SVM - gaussian kernel

gaussian_svm_clf = Pipeline((
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scalar", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", degree=10, coef0=1, C=5)) 
))

# Step 2: Train the model

In [33]:
# training - kernel SVM
gaussian_svm_clf.fit(train_x_std,train_y.values.ravel())

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scalar', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, degree=10))])

# Step 3: Predict for the test points using the model trained in the previous step

In [34]:
# predict - kernel SVM
gauss_svm_pred = gaussian_svm_clf.predict(test_x_std)

# 4.4 - Evaluation


---


# Comparing the accuracies of different SVM models.



In [35]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [69]:
# space for any imports for the following steps
actual_y = test_y.values.tolist()

linear_svm_accuracy = accuracy_metric(actual_y,linear_svm_pred)
poly_svm_accuracy = accuracy_metric(actual_y,poly_svm_pred)
gauss_svm_accuracy = accuracy_metric(actual_y,gauss_svm_pred)

print("------------------------------------------Accuracy------------------------------------------ \n")
print("Accuracy of Linear SVM predictions:\t\t {} %\n".format(linear_svm_accuracy))
print("Accuracy of Polynomial Kernel SVM predictions:\t {} %\n".format(poly_svm_accuracy))
print("Accuracy of Gaussian Kernel SVM predictions:\t {} %\n".format(gauss_svm_accuracy))



------------------------------------------Accuracy------------------------------------------ 

Accuracy of Linear SVM predictions:		 73.02839116719242 %

Accuracy of Polynomial Kernel SVM predictions:	 79.07465825446897 %

Accuracy of Gaussian Kernel SVM predictions:	 78.04942166140904 %



# 5. Neural Networks



---



# 5.1 Single layer neural network


---

# Step 1: Implement a single layer neural network to classify the data points

In [50]:
#import tensorflow as tf

In [51]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [52]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[8])]

dnn_clf = tf.estimator.DNNClassifier(hidden_units=[200], n_classes=2,
                                     feature_columns=feature_cols)

# Step 2: Train the model

In [53]:
input_fn = tf.estimator.inputs.numpy_input_fn( x={"X": train_x_std}, y=train_y.to_numpy(), num_epochs=70, batch_size=50, shuffle=True)
dnn_clf.train(input_fn=input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


# Step 3: Predict for the test points using the model trained in the previous step

In [54]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={"X": test_x_std}, y=test_y.to_numpy(), shuffle=False)

In [56]:
y_pred_iter1 = dnn_clf.predict(input_fn=test_input_fn)
y_pred1 = list(y_pred_iter1)

# 5.2 Multi - Layer neural network

---

# Step 1: Implement a multi - layer neural network to classify the data points


In [58]:
feature_cols2 = [tf.feature_column.numeric_column("X", shape=[8])]

dnn_clf2 = tf.estimator.DNNClassifier(hidden_units=[50, 70], n_classes=2,
                                     feature_columns=feature_cols2, optimizer='Adagrad', activation_fn=tf.nn.relu)


# Step 2: Train the model

In [59]:
input_fn2 = tf.estimator.inputs.numpy_input_fn(x={"X": train_x_std}, y=train_y.to_numpy(), num_epochs=100, batch_size=32, shuffle=True)
dnn_clf2.train(input_fn=input_fn2)

# Step 3: Predict for the test points using the model trained in the previous step

In [60]:
test_input_fn2 = tf.estimator.inputs.numpy_input_fn(
    x={"X": test_x_std}, y=test_y.to_numpy(), shuffle=False)


y_pred_iter2 = dnn_clf2.predict(input_fn=test_input_fn2)
y_pred2 = list(y_pred_iter2)

# 5.3 Multi - Layer neural network

---

# Step 1: Implement a multi - layer neural network to classify the data points




In [62]:
feature_cols3 = [tf.feature_column.numeric_column("X", shape=[8])]

dnn_clf3 = tf.estimator.DNNClassifier(hidden_units=[20, 50, 70], n_classes=2,
                                     feature_columns=feature_cols3, optimizer='Adagrad', activation_fn=tf.nn.relu)


Dense Trial

# Step 2: Train the model

In [63]:
input_fn3 = tf.estimator.inputs.numpy_input_fn(x={"X": train_x_std}, y=train_y.to_numpy(), num_epochs=100, batch_size=70, shuffle=True)
dnn_clf3.train(input_fn=input_fn3)

In [75]:
 y321 = dnn_clf3.train(input_fn=input_fn3)

Instructions for updating:
Use standard file utilities to get mtimes.


In [ ]:
 y321.summary()


# Step 3: Predict for the test points using the model trained in the previous step

In [64]:
test_input_fn3 = tf.estimator.inputs.numpy_input_fn(x={"X": test_x_std}, y=test_y.to_numpy(), shuffle=False)

y_pred_iter3 = dnn_clf3.predict(input_fn=test_input_fn3)

y_pred3 = list(y_pred_iter3)


# 5.4 - Evaluation


---


# Comparing the accuracies of different Neural Network models


In [66]:
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

eval_results2 = dnn_clf2.evaluate(input_fn=test_input_fn2)

eval_results3 = dnn_clf3.evaluate(input_fn=test_input_fn3)

Instructions for updating:
The value of AUC returned by this may race with the update so this is deprecated. Please use tf.keras.metrics.AUC instead.


In [67]:

print("------------------------------------------Accuracy------------------------------------------ \n")

print("Accuracy of 5.1 Single Layer Neural Network: {} %\n".format(eval_results))
print("Accuracy of 5.2 Multi-Layer Nueral Network with 2 Hidden layers: {} %\n".format(eval_results2))
print("Accuracy of 5.3 Multi-Layer Neural Network with 3 Hidden Layers {} %\n".format(eval_results3))

------------------------------------------Accuracy------------------------------------------ 

Accuracy of 5.1 Single Layer Neural Network: {'accuracy': 0.8115142, 'accuracy_baseline': 0.6422187, 'auc': 0.85850257, 'auc_precision_recall': 0.8211372, 'average_loss': 0.42337254, 'label/mean': 0.3577813, 'loss': 53.68364, 'precision': 0.8492408, 'prediction/mean': 0.34311828, 'recall': 0.57531226, 'global_step': 21303} %

Accuracy of 5.2 Multi-Layer Nueral Network with 2 Hidden layers: {'accuracy': 0.8154574, 'accuracy_baseline': 0.6422187, 'auc': 0.8678451, 'auc_precision_recall': 0.8345132, 'average_loss': 0.41012752, 'label/mean': 0.3577813, 'loss': 52.00417, 'precision': 0.8501594, 'prediction/mean': 0.33578587, 'recall': 0.58780307, 'global_step': 47550} %

Accuracy of 5.3 Multi-Layer Neural Network with 3 Hidden Layers {'accuracy': 0.8159832, 'accuracy_baseline': 0.6422187, 'auc': 0.85764724, 'auc_precision_recall': 0.82471716, 'average_loss': 0.4239928, 'label/mean': 0.3577813, 'lo

# Comparing the performances of models under sections 2, 3, 4 and 5 namely perceptron, descent procedure, SVM's and neural networks. 

The maximum accuracies in different Sections:
Perceptron : 64.88 %
Polynomial Kernel SVM : 79.07%
Neural Network: 81.59 %

As you can see that, Neural Network performs the best due to its increased complexity of the model. It learns from the training dataset and passes it to different hidden layers to optimise the classification process and reduce errors in classification. The best performance amongst the Neural Network models was the one with 3 hidden layers with 20, 50, and 70 neurons in the hidden layers. It was run with 100 epochs and with a batch size of 32.

SVM performs similarly to Neural Networks, although slightly performs lessere due to its less complex structure of the algorithm. SVM uses kernal trick to transform the feature space into a higher dimension to classify the datapoints accurately. SVM with the polynomial kernal outperforms other SVM models.

Since the architecture of the perceptron is not complex enough to learn all the information of the features, the classfication accuracy takes a hit.